# [모듈 1.1] Real-Time Endpoint 배포 및 추론
---

본 워크샵의 모든 노트북은 `conda_python3` 추가 패키지를 설치하고 모두 이 커널 에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.

- 1. 환경 셋업
- 2. 배포 준비
- 3. 엔드포인트 생성
- 4. 엔드 포인트 추론
- 5. 로컬 엔드 포인트 삭제



---

### 참고
- 김대근님의 리포에서 SageMaker Hugging Face Inference Toolkit 등을 자세히 설명 함.
    - [Lab 2-1: Deploy Hugging Face Transformers in SageMaker Real-time Endpoint](https://github.com/daekeun-ml/sm-huggingface-kornlp/blob/main/lab_2_serving/1_real_time_endpoint.ipynb)
    
    
- HF 공식 사이트 에서 SM Endpint 를 배포하는 가이드
    - [Deploy models to Amazon SageMaker](https://huggingface.co/docs/sagemaker/inference#deploy-a-🤗-transformers-model-trained-in-sagemaker)
    
    
- SageMaker Hugging Face Inference Toolkit: 
    - https://github.com/aws/sagemaker-huggingface-inference-toolkit    

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [9]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%store -r bucket
%store -r prefix
%store -r artifact_path

In [11]:
### 커스텀 라이브러리
import config 

In [12]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()



In [13]:
import os
import json
import sys
import logging
import boto3
import sagemaker
from sagemaker.huggingface import HuggingFaceModel
from sagemaker import session
from transformers import ElectraConfig
from transformers import (
    ElectraModel, ElectraTokenizer, ElectraForSequenceClassification
)

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(filename='tmp.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# 2. 모델 패키징
- 모델 아티펙트를 다운로드 합니다.
- 다운로드 받은 모델 아티펙트의 압축을 해제하고 모델 가중치인 model.pth 파일을 얻습니다.

In [14]:
import os
import config

model_data_dir = config.model_dir
os.makedirs(model_data_dir, exist_ok=True)
print("model_data_dir: ", model_data_dir)

model_data_dir:  models/nsmc


### 모델 패키징 폴더 정리

In [15]:
%%sh -s {model_data_dir}

model_data_dir=$1

echo $model_data_dir

# 기존 데이터 삭제
rm -rf $model_data_dir/*


models/nsmc


### 모델 및 토큰나이저 아티펙트 다운로드

In [16]:
tokenizer_id = config.tokenizer_id
model_id = config.model_id

print(f"tokenizer_id: {tokenizer_id}")
print(f"model_id: {model_id}")

# Download model and tokenizer
model = ElectraForSequenceClassification.from_pretrained(model_id)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_id)

# os.makedirs(model_dir, exist_ok=True)
model.save_pretrained(model_data_dir)
tokenizer.save_pretrained(model_data_dir)

tokenizer_id: monologg/koelectra-small-v3-discriminator
model_id: monologg/koelectra-small-v3-discriminator


Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized

('models/nsmc/tokenizer_config.json',
 'models/nsmc/special_tokens_map.json',
 'models/nsmc/vocab.txt',
 'models/nsmc/added_tokens.json')

### pre-trained 모델 가중치 삭제

In [17]:
%%sh -s {model_data_dir}

model_data_dir=$1

echo $model_data_dir

# 기존 모델 삭제ㅁ
rm -rf pytorch_model.bin


models/nsmc


### fine-tunnig 된 모델 가중치 다운로드 및 이름 변경

In [18]:
%%sh -s {artifact_path} {model_data_dir}

artifact_path=$1
model_data_dir=$2

echo $artifact_path
echo $model_data_dir


# 모델을 S3에서 로컬로 다운로드
aws s3 cp $artifact_path $model_data_dir

# 모델 다운로드 폴더로 이동
cd $model_data_dir

# 압축 해제
tar -xvf model.tar.gz  

rm -rf model.tar.gz  

# 훈련된 가중치를 약속된 pytorch_model.bin 이름으로 변경ㅁ
mv sentimental-electro-hf.pth pytorch_model.bin

s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-06-05-11-54-33-968/output/model.tar.gz
models/nsmc
download: s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-06-05-11-54-33-968/output/model.tar.gz to models/nsmc/model.tar.gz
sentimental-electro-hf.pth


### 모델 아테펙트를 model.tar.gz 로 압축

모델 파라메터 및 토크나이저를 `model.tar.gz`으로 압축합니다. 압축 파일명은 자유롭게 지정할 수 있으나, 반드시 `tar.gz`로 압축해야 합니다.

In [19]:
model_artifact_name = 'model.tar.gz'
!cd {model_data_dir} && tar -czvf {model_artifact_name} *.*

config.json
pytorch_model.bin
special_tokens_map.json
tokenizer_config.json
vocab.txt


### model.tar.gz 를 S3 에 업로드ㅡ

In [20]:
s3_prefix = prefix
s3_model_path = f's3://{bucket}/{s3_prefix}'
!aws s3 cp {model_data_dir}/{model_artifact_name} {s3_model_path}/{model_artifact_name}

print("s3_model_path: \n", s3_model_path)

upload: models/nsmc/model.tar.gz to s3://sagemaker-us-east-1-057716757052/KoElectra-HF/model.tar.gz
s3_model_path: 
 s3://sagemaker-us-east-1-057716757052/KoElectra-HF


모델 파라메터 및 토크나이저를 `model.tar.gz`으로 압축합니다. 압축 파일명은 자유롭게 지정할 수 있으나, 반드시 `tar.gz`로 압축해야 합니다.

# 3. SageMaker Endpoint Deployment

## 3.1. 앤드포인트 배포
 
아래 코드를 보시면 아시겠지만, 지속적으로 업데이트되는 파이썬 버전&프레임워크 버전&트랜스포머 버전에 쉽게 대응할 수 있습니다. AWS에서 관리하고 있는 딥러닝 컨테이너(DLC) 목록을 아래 주소에서 확인해 보세요.

- https://github.com/aws/deep-learning-containers/blob/master/available_images.md

In [21]:
# local_model_path = os.path.join(config.model_dir, 'model.tar.gz')
# print("local_model_path: ", local_model_path)

In [22]:
instance_type = "ml.m5.xlarge"

In [23]:
# create Hugging Face Model Class
hf_model = HuggingFaceModel(
    model_data=f"{s3_model_path}/{model_artifact_name}",  # path to your trained SageMaker model
    role=role,                                            # IAM role with permissions to create an endpoint
#     transformers_version="4.17.0",                        # Transformers version used
#     pytorch_version="1.9.1",                              # PyTorch version used
#     py_version='py38',                                    # Python version used
    transformers_version="4.12.3",                        # Transformers version used
    pytorch_version="1.9.1",                              # PyTorch version used
    py_version='py38',                                    # Python version used
    
)

# deploy model to SageMaker Inference
hf_predictor = hf_model.deploy(
    initial_instance_count=1,
    instance_type= instance_type,
    wait=False
)

[{session.py:2668} INFO - Creating model with name: huggingface-pytorch-inference-2022-06-06-02-48-36-525
[{session.py:3585} INFO - Creating endpoint-config with name huggingface-pytorch-inference-2022-06-06-02-48-36-821
[{session.py:3053} INFO - Creating endpoint with name huggingface-pytorch-inference-2022-06-06-02-48-36-821


In [24]:
%%time 
sess.wait_for_endpoint(hf_predictor.endpoint_name, poll=5)

--------------------------------------------------------------------!CPU times: user 295 ms, sys: 32.8 ms, total: 327 ms
Wall time: 5min 43s


{'EndpointName': 'huggingface-pytorch-inference-2022-06-06-02-48-36-821',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:057716757052:endpoint/huggingface-pytorch-inference-2022-06-06-02-48-36-821',
 'EndpointConfigName': 'huggingface-pytorch-inference-2022-06-06-02-48-36-821',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.9.1-transformers4.12.3-cpu-py38-ubuntu20.04',
     'ResolvedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference@sha256:13a97f950cc426afa114b777d443e2f9c6292ac169b65340e59a86ee5315cef1',
     'ResolutionTime': datetime.datetime(2022, 6, 6, 2, 49, 25, 659000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2022, 6, 6, 2, 48, 37, 55000, tzinfo=tzlocal()),
 'LastModif

## 3.2. Wait for the endpoint jobs to complete

엔드포인트가 생성될 때까지 기다립니다. 약 5-10분의 시간이 소요됩니다.

In [25]:
from IPython.core.display import display, HTML

def make_endpoint_link(region, endpoint_name, endpoint_task):
    
    endpoint_link = f'<b><a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">{endpoint_task} Review Endpoint</a></b>'   
    return endpoint_link 
        
endpoint_link1 = make_endpoint_link(region, hf_predictor.endpoint_name, '[Deploy model from S3]')


display(HTML(endpoint_link1))

<br>

# 4. Inference

---

두 개의 엔드포인트가 배포되었습니다. 샘플 데이터로 직접 추론을 수행해 봅니다.

## 4.1. SageMaker Python SDK 추론

In [26]:
# example request, you always need to define "inputs"
payload = {
   "inputs": [
       "정말 재미있습니다. 세 번 봐도 질리지 않아요.",
       "시간이 아깝습니다. 다른 영화를 보세요."
   ]
}

In [27]:
hf_predictor.predict(payload)

[{'label': 'LABEL_1', 'score': 0.9441463351249695},
 {'label': 'LABEL_0', 'score': 0.6941782832145691}]

## 4.2. Botoe3 추론

In [28]:
import boto3

runtime_client = boto3.Session().client('sagemaker-runtime')


In [29]:
import time
import json
from inference_utils import invoke_endpoint_hf
payload_dump = json.dumps(payload)

start_time = time.time()
result = invoke_endpoint_hf(runtime_client, hf_predictor.endpoint_name, 
                         payload_dump,
                         content_type='application/json'
                        )

print("--- %s seconds ---" % (time.time() - start_time))
print('result: ', result)

--- 0.24317169189453125 seconds ---
result:  [{'label': 'LABEL_1', 'score': 0.9441463351249695}, {'label': 'LABEL_0', 'score': 0.6941782832145691}]


# 5. 엔드 포인트 삭제
- 기존에 생성한 로컬 세이지 메이커 모델, 앤드포인트 컨피그, 앤드포인트 삭제

In [30]:
from inference_utils import delete_endpoint

client = boto3.Session().client('sagemaker')
delete_endpoint(client, hf_predictor.endpoint_name)

--- Deleted model: huggingface-pytorch-inference-2022-06-06-02-48-36-525
--- Deleted endpoint: huggingface-pytorch-inference-2022-06-06-02-48-36-821
--- Deleted endpoint_config: huggingface-pytorch-inference-2022-06-06-02-48-36-821
